In [1]:
import pandas as pd
import ast
import datetime
import json

# Load books info from csv files

After checking the dfs, we can find that in "bk_2019","bk_2020" and "bk_2021",the format of the cells in column "genre" looks like list but not actual list. So first of all, we need to convert them to list and then to strings.

In [2]:
# lists = {}
# lists['genre']=bk_2019['genre']
# df = pd.DataFrame(lists)
# # convert the string representation of lists into actual lists
# df['genre'] = df['genre'].apply(ast.literal_eval)
# # join the lists into clean string
# df['genre'] = df['genre'].apply(lambda x: ', '.join(x))
# df
# # print(df['genre'].apply(type))

In [3]:
bk_2019 = pd.read_csv("../data/raw_data/books_2019_df.csv")
# convert the string representation of lists into actual lists
bk_2019['genre'] = bk_2019['genre'].apply(ast.literal_eval)
# join the lists into clean string
bk_2019['genre'] = bk_2019['genre'].apply(lambda x: ', '.join(x))
bk_2019.head(3)
bk_2019.shape

(400, 12)

In [4]:
bk_2020 = pd.read_csv("../data/raw_data/books_2020_df.csv")

bk_2020['genre'] = bk_2020['genre'].apply(ast.literal_eval)
bk_2020['genre'] = bk_2020['genre'].apply(lambda x: ', '.join(x))
bk_2020.shape

(399, 12)

In [5]:
bk_2021 = pd.read_csv("../data/raw_data/books_2021_df.csv")

bk_2021['genre'] = bk_2021['genre'].apply(ast.literal_eval)
bk_2021['genre'] = bk_2021['genre'].apply(lambda x: ', '.join(x))
bk_2021.shape

(340, 12)

In [6]:
bk_2022 = pd.read_csv("../data/raw_data/books_2022_df.csv")
bk_2022.shape

(340, 12)

In [7]:
bk_2023 = pd.read_csv("../data/raw_data/books_2023_df.csv")
bk_2023.shape

(299, 12)

# Add column "year" for each df

In [8]:
bk_2019['year'] = 2019
bk_2020['year'] = 2020
bk_2021['year'] = 2021
bk_2022['year'] = 2022
bk_2023['year'] = 2023

# Add column "award" for each df

In [9]:
# load award_category.json
with open("../data/raw_data/award_category.json", 'r') as file:
    category = json.load(file)

In [10]:
award_count_2019 = category['2019']
award_count_2020 = category['2020']
award_count_2021 = category['2021']
award_count_2022 = category['2022']
award_count_2023 = category['2023']

In [11]:
def add_award(award_counts):
    current_index = 0
    categories = []
    for category, count in award_counts.items():
        categories += [category] * count  # Extend the list by repeating the category 'count' times
        current_index += count
    return categories

In [12]:
bk_2019['award'] = add_award(award_count_2019)
bk_2020['award'] = add_award(award_count_2020)
bk_2021['award'] = add_award(award_count_2021)
bk_2022['award'] = add_award(award_count_2022)
bk_2023['award'] = add_award(award_count_2023)

# Create an ensemble df

In [13]:
df = pd.concat([bk_2019,bk_2020,bk_2021,bk_2022,bk_2023], ignore_index=True)
df.shape

(1778, 14)

In [14]:
df.columns

Index(['title', 'ISBN', 'author', 'genre', 'rating', 'number_of_ratings',
       'number_of_reviews', 'pages', 'format', 'publish info', 'description',
       'Votes', 'year', 'award'],
      dtype='object')

In [15]:
df.isna().sum()

title                0
ISBN                 6
author               0
genre                0
rating               0
number_of_ratings    0
number_of_reviews    0
pages                8
format               0
publish info         0
description          0
Votes                0
year                 0
award                0
dtype: int64

In [16]:
df.duplicated().sum()

0

In [17]:
df.nunique()

title                1687
ISBN                   15
author               1307
genre                1605
rating                241
number_of_ratings    1685
number_of_reviews    1616
pages                 546
format                  8
publish info          411
description          1690
Votes                1691
year                    5
award                  89
dtype: int64

# Cleaning Dataframe

- drop column "ISBN"
- We can see that most of the cells filled in the "ISBN not found", so we can drop it.

In [18]:
df = df.drop(columns=['ISBN'])

In [19]:
df.head(3)

,title,author,genre,rating,number_of_ratings,number_of_reviews,pages,format,publish info,description,Votes,year,award
0,The Testaments,Margaret Atwood,"Fiction, Dystopia, Science Fiction, Feminism, ...",4.19,352440,31334,422.0,Hardcover,"First published September 10, 2019",When the van door slammed on Offred's future a...,98291,2019,best-fiction-books-2019
1,Normal People,Sally Rooney,"Fiction, Romance, Contemporary, Literary Ficti...",3.81,1388906,135364,273.0,Hardcover,"First published August 28, 2018",At school Connell and Marianne pretend not to ...,40081,2019,best-fiction-books-2019
2,Where the Forest Meets the Stars,Glendy Vanderah,"Fiction, Contemporary, Mystery, Romance, Fanta...",4.14,192265,16504,332.0,Hardcover,"First published March 1, 2019","In this gorgeously stunning debut, a mysteriou...",29342,2019,best-fiction-books-2019


- deal with column "genre"
- extract the value of column and create a new df about genre of books

In [20]:
# split the 'genre' strings into lists of individual genres
genres_list = df['genre'].str.split(', ').explode().unique()

# create a new DataFrame with unique genres as a category
genres_df = pd.DataFrame(genres_list, columns=['genre'])

# looks like there is an empty string, convert it to null value
genres_df= genres_df.replace('', pd.NA).dropna(subset=['genre'])

genres_df['id'] = range(1, len(genres_df) + 1)

# change the order of columns
genres_df = genres_df[['id','genre']]

# export as csv file
genres_df.to_csv('../data/database/book_genres.csv', index=False)

- deal with column "publish info"
- keep only the publish date and convert it to datetime type

In [21]:
df['publish_date'] = df['publish info'].str.extract(r'(\w+ \d{1,2}, \d{4})')
df['publish_date'] = pd.to_datetime(df['publish_date'])

# drop "publish info"
df = df.drop(columns=['publish info'])

- drop the rows with "not found"

In [22]:
df_filtered = df[df['number_of_ratings'] != 0]
df_filtered.head(3)

,title,author,genre,rating,number_of_ratings,number_of_reviews,pages,format,description,Votes,year,award,publish_date
0,The Testaments,Margaret Atwood,"Fiction, Dystopia, Science Fiction, Feminism, ...",4.19,352440,31334,422.0,Hardcover,When the van door slammed on Offred's future a...,98291,2019,best-fiction-books-2019,2019-09-10
1,Normal People,Sally Rooney,"Fiction, Romance, Contemporary, Literary Ficti...",3.81,1388906,135364,273.0,Hardcover,At school Connell and Marianne pretend not to ...,40081,2019,best-fiction-books-2019,2018-08-28
2,Where the Forest Meets the Stars,Glendy Vanderah,"Fiction, Contemporary, Mystery, Romance, Fanta...",4.14,192265,16504,332.0,Hardcover,"In this gorgeously stunning debut, a mysteriou...",29342,2019,best-fiction-books-2019,2019-03-01


- format the names of columns

In [23]:
# change the order of columns
df_filtered = df_filtered[['title','author','genre','publish_date','pages','format','rating','number_of_ratings','number_of_reviews','Votes','year','award','description']]

new_columns_name = {
    'number_of_ratings':'rating_couunts',
    'number_of_reviews':'reviews_counts',
    'Votes':'votes'
}

df_filtered = df_filtered.rename(columns=new_columns_name)
df_filtered.head(3)

,title,author,genre,publish_date,pages,format,rating,rating_couunts,reviews_counts,votes,year,award,description
0,The Testaments,Margaret Atwood,"Fiction, Dystopia, Science Fiction, Feminism, ...",2019-09-10,422.0,Hardcover,4.19,352440,31334,98291,2019,best-fiction-books-2019,When the van door slammed on Offred's future a...
1,Normal People,Sally Rooney,"Fiction, Romance, Contemporary, Literary Ficti...",2018-08-28,273.0,Hardcover,3.81,1388906,135364,40081,2019,best-fiction-books-2019,At school Connell and Marianne pretend not to ...
2,Where the Forest Meets the Stars,Glendy Vanderah,"Fiction, Contemporary, Mystery, Romance, Fanta...",2019-03-01,332.0,Hardcover,4.14,192265,16504,29342,2019,best-fiction-books-2019,"In this gorgeously stunning debut, a mysteriou..."


In [24]:
# export as csv file
df_filtered.to_csv('../data/database/book_info.csv', index=False)